# Biol 359A  | Principal Component Analysis
### Spring 2023, Week 9
<hr>
Learning Objectives:

  - Understand motivations of dimension reduction
  - Discuss applications towards data visualization and data exploration
  - Interpret the results of a principal component analysis


In [ ]:
!git clone https://github.com/BIOL359A-FoundationsOfQBio-Spr23/week9_pca
!mkdir ./data
!cp week9_pca/data/* ./data
!pip install palmerpenguins

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns 
import sklearn as sk
import matplotlib.pyplot as plt

from palmerpenguins import load_penguins

from sklearn.decomposition import PCA
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler

%matplotlib inline

def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

TITLE_FONT = 20
LABEL_FONT = 16
TICK_FONT = 16
FIG_SIZE = (8,8)
COLORS= ["#008080","#CA562C"]

sns.set(font_scale=1.5, rc={'figure.figsize':FIG_SIZE}) 
sns.set_style()
plt.rc("axes.spines", top=False, right=False)


In [ ]:
def colorizer(x, y):
    """
    Map x-y coordinates to a rgb color
    """
    r = min(1, 1+y/3)
    b = min(1, 1-y/3)
    g = 1/4 + x/16
    return (r, g, b)

def gen_basic_plot(A, eigen=False):
    xvals = np.linspace(-4, 4, 9)
    yvals = np.linspace(-3, 3, 7)
    xygrid = np.column_stack([[x, y] for x in xvals for y in yvals])

    uvgrid = np.dot(A, xygrid)
    # Map grid coordinates to colors
    colors = list(map(colorizer, xygrid[0], xygrid[1]))

    # Plot grid points 
    plt.scatter(xygrid[0], xygrid[1], s=40, c=colors, edgecolor="none")
    # Set axis limits
    plt.grid(True)
    plt.axis("equal")
    plt.title("Original Grid")
    if eigen:
        eigen_values, eigen_vectors = np.linalg.eig(A)
    
        eig_vec1 = eigen_vectors[:,0]
        eig_vec2 = eigen_vectors[:,1]
        np.set_printoptions(precision=3)
        print(f"Eigen Vector: {eig_vec1} - Eigen Value: {eigen_values[0]:.2f}")
        print(f"Eigen Vector: {eig_vec2} - Eigen Value: {eigen_values[1]:.2f}")
        origin = [0,0]
        plt.quiver(*origin, *eig_vec1, color=['r'], scale=21)
        plt.quiver(*origin, *eig_vec2, color=['b'], scale=21) 


    plt.show()
    plt.scatter(uvgrid[0], uvgrid[1], s=40, c=colors, edgecolor="none")
    # Set axis limits
    plt.grid(True)
    plt.title("Transformed Grid")
    if eigen:
        plt.quiver(*origin, *eig_vec1, color=['r'], scale=21)
        plt.quiver(*origin, *eig_vec2, color=['b'], scale=21)
    plt.axis("equal")
    plt.show()
    

def plot_wrapper(a=2,b=1,c=-1,d=1, eigen=False):
    A = np.column_stack([[a, b], [c, d]])
    gen_basic_plot(A, eigen=eigen)

# Linear Algebra and PCA Example (Questions 1-3)

We are going to start with the following matrix:

$$\begin{bmatrix} 2 & 1 \\ -1 & 1 \\ \end{bmatrix}$$

(This matrix has no real eigen vectors)

In [ ]:
plot_wrapper(a=2,b=1,c=-1,d=1, eigen=False)

This matrix has real eigen vectors:

$$\begin{bmatrix} 5 & 1 \\ 4 & 2 \\ \end{bmatrix}$$

In [ ]:
plot_wrapper(a=5, b=1, c=4,d=2, eigen=True)

Introducing the Covariance Matrix:

$$ var(x) = E[(x-\mu_{x})^2] $$
Recall when we defined the covariance:

$$ cov(x,y) = E[x-\mu_{x}]E[y-\mu_{y}] $$

So for example, If I had a data matrix, with N observations, and 3 features - a, b, and c - I will define my covariance matrix $\Sigma$ as:
 
$$ \Sigma = \begin{bmatrix} var(a) & cov(a,b) & cov(a,c)\\ cov(b,a) & var(b)& cov(b,c) \\ cov(c,a) & cov(c,b) & var(c) \end{bmatrix}$$

Which we usually estimate with:

$$ \frac{1}{N-1} XX^T = \hat{\Sigma} $$

(Look familiar? This is the outer product with a normalization factor!)

In [ ]:
def demonstrate_pca():
    # Sample data
    np.random.seed(42)
    x = np.random.normal(0, 1, 100)
    y = 2 * x + np.random.normal(0, 1, 100)
    data = np.vstack((x, y)).T

    # Centering the data
    centered_data = data - np.mean(data, axis=0)

    # Performing PCA
    cov_matrix = np.cov(centered_data, rowvar=False)
    eig_values, eig_vectors = np.linalg.eig(cov_matrix)

    # Sorting eigenvalues and eigenvectors in descending order
    sorted_indices = np.argsort(eig_values)[::-1]
    eig_values = eig_values[sorted_indices]
    eig_vectors = eig_vectors[:, sorted_indices]

    # Plotting the data
    plt.scatter(centered_data[:, 0], centered_data[:, 1], alpha=0.5, label='Data')

    # Plotting the best PCA direction
    best_direction = eig_vectors[:, 0]
    projected_data = np.dot(centered_data, best_direction)
    max_value = np.max(projected_data)
    min_value = np.min(projected_data)
    t = np.linspace(min_value, max_value, 100)
    plt.plot(t * best_direction[0], t * best_direction[1], color='red', label='Best PCA')

    # Plotting a suboptimal direction
    suboptimal_direction = eig_vectors[:, 1]
    projected_data = np.dot(centered_data, suboptimal_direction)
    max_value = np.max(projected_data)
    min_value = np.min(projected_data)
    t = np.linspace(min_value, max_value, 100)
    plt.plot(t * suboptimal_direction[0], t * suboptimal_direction[1], color='green', label='Suboptimal PCA')

    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('PCA Demonstration')
    plt.legend()
    plt.grid(True)
    plt.axis('equal')
    plt.show()

# Calling the function
demonstrate_pca()

## Generate a toy dataset

Notice the direction where the most variance occurs. This line will be slightly different from our linear regression line, because in PCA we're treating these variables the same. (We care about the error in both the x and y direction, rather than simply the y-direction).

We then generate the eigen vector of our covariance matrix, and plot it over the original data. 

In [ ]:
def example_pca():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    plt.scatter(X[:, 0], X[:, 1])
    plt.axis('equal')
    plt.show()
    pca = PCA(n_components=2)
    pca.fit(X)
    
    # plot data
    plt.scatter(X[:, 0], X[:, 1], alpha=0.2)
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector * 3 * np.sqrt(length)
        draw_vector(pca.mean_, pca.mean_ + v)
    plt.axis('equal'); 
    plt.show()
    
def draw_vector(v0, v1, ax=None):
    ax = ax or plt.gca()
    arrowprops=dict(arrowstyle='->',
                    color='k',
                    linewidth=2,
                    shrinkA=0, shrinkB=0)
    ax.annotate('', v1, v0, arrowprops=arrowprops)

example_pca()

### Coordinate System
Now that we have the eigenvectors, we can do two things with them.
First, we're going to look at using them as a new coordinate system.

In [ ]:
def coordinatesystem_example():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    pca = PCA(n_components=2, whiten=True)
    pca.fit(X)

    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.15)

    # plot data
    ax[0].scatter(X[:, 0], X[:, 1], alpha=0.2)
    for length, vector in zip(pca.explained_variance_, pca.components_):
        v = vector * 3 * np.sqrt(length)
        draw_vector(pca.mean_, pca.mean_ + v, ax=ax[0])
    ax[0].axis('equal');
    ax[0].set(xlabel='x', ylabel='y', title='input')

    # plot principal components
    X_pca = pca.transform(X)
    ax[1].scatter(X_pca[:, 0], X_pca[:, 1], alpha=0.2)
    draw_vector([0, 0], [0, 3], ax=ax[1])
    draw_vector([0, 0], [3, 0], ax=ax[1])
    ax[1].axis('equal')
    ax[1].set(xlabel='component 1', ylabel='component 2',
              title='principal components',
              xlim=(-5, 5), ylim=(-3, 3.1))
coordinatesystem_example()

### Projection
The other option we have is to simply use one principal component, and represent our data with one vector rather than two. What does this accomplish? 

In [ ]:
def projection_example():
    rng = np.random.RandomState(1)
    X = np.dot(rng.rand(2, 2), rng.randn(2, 200)).T
    pca = PCA(n_components=2, whiten=True)
    pca.fit(X)

    fig, ax = plt.subplots(1, 2, figsize=(15, 6))
    fig.subplots_adjust(left=0.0625, right=0.95, wspace=0.15)

    # plot data
    ax[0].scatter(X[:, 0], X[:, 1], alpha=0.5)
    ax[0].axis('equal');
    ax[0].set(xlabel='x', ylabel='y', title='input')

    # plot principal components
    pca = PCA(n_components=1, whiten=True)
    pca.fit(X)
    X_pca = pca.transform(X)
    X_new = pca.inverse_transform(X_pca)
    ax[1].scatter(X[:, 0], X[:, 1], alpha=0.2)
    ax[1].scatter(X_new[:, 0], X_new[:, 1], color="blue", alpha=0.8)
    ax[1].axis('equal')
    ax[1].set(xlabel='x', ylabel='y',
              title='projected data (First PC)')
    
projection_example()

In [ ]:
def PerformPCA(X, n_dimensions=10, plot=True):
    """
    Uses sklearn PCA tool to perform PCA
    input:
    X: Pandas Dataframe or Numpy Array of features
    n_dimensions: Number of PCs to fit
    
    output:
    X_pca: Pandas dataframe with column titles of PC1,...,PCn
    """
    X_standardized = StandardScaler().fit_transform(X)
    pca = PCA(n_components=n_dimensions)
    pca.fit(X_standardized)
    X_pca_array = pca.transform(X_standardized)
    column_names = ['PC{}'.format(i+1) for i in range(n_dimensions)] 
    X_pca = pd.DataFrame(X_pca_array, columns=column_names)
    if plot:
        plt.plot(column_names, np.cumsum(pca.explained_variance_ratio_), 'o--')
        plt.title('Skree Plot', fontsize=TITLE_FONT)
        plt.xlabel('Number of PCs', fontsize=LABEL_FONT)
        plt.ylabel('Total Percent Variance Explained', fontsize=LABEL_FONT)
        plt.ylim(0,1)
        plt.show()
    return X_pca

# Now we are going to explore a couple datasets that we've looked at before with PCA (Question 4 + 5)
The first dataset we'll look at is penguin characterstics

In [ ]:
penguins=load_penguins()
penguins.dropna(inplace=True)
features=["bill_length_mm", "bill_depth_mm", "flipper_length_mm","body_mass_g"]
label='species'
sns.pairplot(penguins, vars=features, corner=True, hue="species", markers=["o", "s", "D"])

In [ ]:
penguins_X = penguins[features]
penguins_y = penguins[label]
penguins_PCA_df = PerformPCA(penguins_X, n_dimensions=4)
penguins_PCA_df = penguins_PCA_df.join(penguins_y)
sns.scatterplot(x='PC1',y='PC2',hue='species',data=penguins_PCA_df,style='species',markers=["o", "s", "D"])
plt.show()

Looking at the penguin plot of PC1 vs. PC2, we can see that we can visualize and seperate the penguins reasonably well even though we went from 4 features (bill length, bill depth, flipper length, body mass) down to 2 features on the chart (PC1, PC2). 

In [ ]:
sns.boxplot(x="species", y="PC1", hue="species",
                 data=penguins_PCA_df)

In [ ]:
sns.boxplot(x="species", y="PC2", hue="species",
                 data=penguins_PCA_df)

# We are going to look at our trusty breast cancer dataset (Question 6 + 7)

Remember, we could build fairly effective classification models with this dataset.

In [ ]:
cancer_raw = load_breast_cancer()
print(cancer_raw.DESCR)
tumor_features = pd.DataFrame(cancer_raw.data, columns=cancer_raw.feature_names)
tumor = pd.DataFrame(cancer_raw.target, columns=['tumor'])
tumor_nominal = tumor.replace({'tumor': {0: 'benign', 1: 'malignant'}})
cancer_df = pd.concat([tumor_features, tumor_nominal], axis=1)
tumor_features.describe()

In [ ]:
PCA_df = PerformPCA(tumor_features)
PCA_df = PCA_df.join(tumor_nominal)
sns.scatterplot(x='PC1',y='PC2',style='tumor',hue='tumor',data=PCA_df,markers=True)
plt.show()

But if I plot different PCs...

In [ ]:
sns.scatterplot(x='PC4',y='PC5',style='tumor',hue='tumor',data=PCA_df,markers=True)
plt.show()